In [2]:
# Load Trimet JSON into dataframe: 

import pandas as pd
df = pd.read_json('data-archive2023-04-21.json')
df.head(5)

,EVENT_NO_TRIP,EVENT_NO_STOP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,VELOCITY,DIRECTION,RADIO_QUALITY,GPS_LONGITUDE
0,218523654,218523719,13-Dec-22,3604,138688,42869,-122.563768,45.530158,12.0,0.8
1,218523654,218523719,13-Dec-22,3604,138728,42875,-122.563898,45.530517,12.0,0.9
2,218523654,218523719,13-Dec-22,3604,138756,42880,-122.563973,45.53077,12.0,0.8
3,218523654,218523719,13-Dec-22,3604,138779,42885,-122.564017,45.530988,12.0,0.8
4,218523654,218523720,13-Dec-22,3604,138796,42930,-122.56396,45.531145,12.0,0.8


In [5]:
df.describe()


,EVENT_NO_TRIP,EVENT_NO_STOP,VEHICLE_ID,METERS,ACT_TIME
count,2.227713e+06,2.227713e+06,2.227713e+06,2.227713e+06,2.227713e+06
mean,2.190073e+08,2.190073e+08,3.783808e+04,1.299700e+05,5.065729e+04
std,2.622959e+05,2.622986e+05,8.010238e+05,8.646476e+04,1.856872e+04
min,2.184721e+08,2.184721e+08,2.901000e+03,0.000000e+00,1.261800e+04
25%,2.188933e+08,2.188933e+08,3.167000e+03,5.950200e+04,3.451800e+04
50%,2.189842e+08,2.189843e+08,3.531000e+03,1.174280e+05,5.119800e+04
75%,2.190761e+08,2.190762e+08,3.749000e+03,1.871000e+05,6.496700e+04
max,2.201692e+08,2.201693e+08,1.872430e+07,4.907190e+05,9.775600e+04


In [9]:
# 1. There are no 0/NaN values (each row has all values)

notna = df.notna()
if False in notna: 
    print ('False')
else: 
    print ("No NaN/empty values found")

No NaN/empty values found


In [12]:
# 2. The velocity is never greater than 40(m/s) (~= 90mph) 
# ***assuming velocity is actually radio quality right now: fix 
vel = df['RADIO_QUALITY']
vel.describe()


count     2227713
unique         11
top          12.0
freq      1569101
Name: RADIO_QUALITY, dtype: object

In [ ]:
# 3. A trip should have a trip number as well as a vehicle number. 
# The code for #1 should cover this 

In [30]:
# 4. Every same trip number should have the same vehicle number 
# trips = df.groupby(['EVENT_NO_TRIP']).mean()
trips = df['EVENT_NO_TRIP'].unique()
trips.size


5751

In [34]:
# 5. There are a few millions records daily but not more (>5mil = alarm)
df['EVENT_NO_TRIP'].size

2227713

In [36]:
# 6. The date should always be the same in every row 
date = df['OPD_DATE'].unique()

array(['13-Dec-22'], dtype=object)

In [ ]:
# 7. The velocity should conform to a (normal?) distribution - max/min/mean speeds 
# Can't do without knowing what velocity column actually is!  